In [3]:
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import PeftModel, PeftConfig
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [ ]:
from huggingface_hub import notebook_login

# notebook_login()

In [5]:
# Method and Model Configuration
entity = "rstern"
retrain_from_checkpoint = True
model_config = "org_model" # possible values: org_model, frozen
checkpoint = "checkpoint-810"
author_label_only = False
augmented_data = True
experiment_name = "org_aug_b" # org_org_a org_aug_b
debugg = False
# model name
model_name = 'mistralai/Mistral-7B-v0.1'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# new tokens
new_tokens = ["[LP2]"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
tokenizer.pad_token = tokenizer.eos_token
    
config = PeftConfig.from_pretrained(f'multilabel_classification/{model_config}/{checkpoint}')
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
lora_model = PeftModel.from_pretrained(model, f'multilabel_classification/{model_config}/{checkpoint}', is_trainable=True)

# there are many errors when pushing to the hub, as the embedding layer sizes mismatch.
# lora_model.push_to_hub("mistral-lp2-test-model", tokenizer=tokenizer, config=config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/579M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Stern5497/mistral-lp2-test-model/commit/1ccfad53d74aab6dfa78f67bcf6cdd1593d4317d', commit_message='Upload model', commit_description='', oid='1ccfad53d74aab6dfa78f67bcf6cdd1593d4317d', pr_url=None, pr_revision=None, pr_num=None)